# modisco formatting
4/12/24

This file focuses on both testing and formatting the npy files to concatenate a single celltype and then we will test the file with tfmodisco to see if it works

First doing it with multitasking because of potential speed? also no dealing with cell type specific token

In [4]:
#the first step is creating the proper sized matrix
#for this we will only use the train data (as the val and test are still running). We have a total of 16792/2 for multitasking or ctst
#we will do ctst as that should have results that make more sense!
import numpy as np
npz = np.zeros((16792//2,4,1023), dtype=np.float16) #can do float 16 since it already is!
print(npz.shape)

(8396, 4, 1023)


In [26]:
npz_onehot = np.zeros((16792//2,4,1023), dtype=np.int16) #can do 
print(npz_onehot.shape)

(8396, 4, 1023)


In [27]:
#let's compute the size in GB of this matrix
print(npz.nbytes/1024/1024/1024) #actually not an issue at all!
print(npz_onehot.nbytes/1024/1024/1024) #actually not an issue at all! same as above

0.06399384140968323
0.06399384140968323


In [8]:
#now we need to load in the data one at a time, and one hot encode it then we can save it to the matrix
#first we modified ism_utils to be the proper shape that it returns
from shap_analysis.ism_utils import ISMUtils
multitasking_path1 = '/data/leslie/sarthak/hyena/hyena-dna/outputs/2024-03-27/18-39-11-031863/checkpoints/25-val_loss=0.52186.ckpt' #the 25 epoch one
a = np.load('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/multitasking_25epoch_174906_reg.npy')
utils = ISMUtils('DNase_allcelltypes', multitasking_path1, classification=True)
out = utils.onehot_ism(174906,ism=a)
print(out[0].shape) #shoudl be 161 x 4 x 1023

(161, 4, 1023)


In [20]:
#the benefit of onehot_ism is that it can load it in on its own, we do need the files to still access the CRE tho
#also let's test to see if this one hot encoded actually makes any sense
onehot = out[1]
print(onehot[1,:,:10])
print(out[2])
#looks right
np.set_printoptions(precision=3, suppress=True)
print(out[0][1,:,:10]) #things seem to be fine and line up!

[[1. 1. 0. 1. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]]
AAGAACTTCAAAAGCTCCATGACCAAATTATTTTATGAGAACCCCAAGTGAACACATTTGGCTTTGAAACAACCATCAGTGATTTGATGGTTTACTAATTTTCAAAGAAATTACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATTTGCAGGATAGTCTGTGAGATGAAACTGGTTAAGGATTAAGAAGAACAGGGTTCTAGTGACAACTCTTTGGATAAGCTGTTTTGTTTCTCTCAACTTCAGTCACTGCAGCTGCAGAATGAAAGCAGAGAAGTAACTGAAGTAATTCAATGATTCTTACTTCTTACAACCTCAGAAAGTCTACAAATCGATGAATATAAGTTTTGGAGTGTTACTATTACAAACAGTGTCTTAATAAACACCCTTAAACATGTCTCCATGTGTGTTTTAGTGTTTCTCTTGGAAAGTGTTGATTGCGCCTCTTTAGAAAATTGTGAAATAAATGTAGTCACCACCAACATATTTTAAGAAAGATGATTGAGCAATACAAAAAATACAACTGTATTACATAGAGTTAAGGATAATGAATGTTTCATGAAATTAAGTTTTGTTTTATTTATAAGGGCATATGTACATATATATGTACTGAGTTGTTTTTTAAGAGGTATTTTCTACTGTGAGTTATTTAAAGACAGTTTGAAATACACTCTACTGACAGTAATTATCTAAGATGAACTTGATGAGTCATATGCTTTCTTCATATTTAACTTCATTAGGTATGGCAAAATTTCCTCCATAGTTGGCAGTATCAACTTACACTCTCATCATCTGAATTTGAGAGCCTGGATCTTCTCTAAAATTC

In [25]:
#just to verify, let's check to see the 153rd column
print(onehot[153,:,:10])
print(out[2])
print(out[0][153,:,:10]*10) #things seem the same and consistent, just uses more decimal places so can't tell some of them, so multiply by 10 to make sure
#just make sure that the -0 is actually 0
print(out[0][153,0,2] == 0)

[[1. 1. 0. 1. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]]
AAGAACTTCAAAAGCTCCATGACCAAATTATTTTATGAGAACCCCAAGTGAACACATTTGGCTTTGAAACAACCATCAGTGATTTGATGGTTTACTAATTTTCAAAGAAATTACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATACAATTTGCAGGATAGTCTGTGAGATGAAACTGGTTAAGGATTAAGAAGAACAGGGTTCTAGTGACAACTCTTTGGATAAGCTGTTTTGTTTCTCTCAACTTCAGTCACTGCAGCTGCAGAATGAAAGCAGAGAAGTAACTGAAGTAATTCAATGATTCTTACTTCTTACAACCTCAGAAAGTCTACAAATCGATGAATATAAGTTTTGGAGTGTTACTATTACAAACAGTGTCTTAATAAACACCCTTAAACATGTCTCCATGTGTGTTTTAGTGTTTCTCTTGGAAAGTGTTGATTGCGCCTCTTTAGAAAATTGTGAAATAAATGTAGTCACCACCAACATATTTTAAGAAAGATGATTGAGCAATACAAAAAATACAACTGTATTACATAGAGTTAAGGATAATGAATGTTTCATGAAATTAAGTTTTGTTTTATTTATAAGGGCATATGTACATATATATGTACTGAGTTGTTTTTTAAGAGGTATTTTCTACTGTGAGTTATTTAAAGACAGTTTGAAATACACTCTACTGACAGTAATTATCTAAGATGAACTTGATGAGTCATATGCTTTCTTCATATTTAACTTCATTAGGTATGGCAAAATTTCCTCCATAGTTGGCAGTATCAACTTACACTCTCATCATCTGAATTTGAGAGCCTGGATCTTCTCTAAAATTC

In [35]:
#the final test
print(np.allclose(out[0],out[0]*out[1])) #yeah it's identical cuz one hot encoded is all!

True


In [28]:
#now we can actually use it and make a matrix out of it!
#for now say celltype = 0
celltype = 0
npz[celltype] = out[0][celltype]
npz_onehot[celltype] = out[1][celltype]

In [36]:
print(npz[celltype,:,:10]) #indeed looks like the one hot values!
print(npz_onehot[celltype,:,:10]) #same as above as we expected
print(np.allclose(npz[celltype],npz_onehot[celltype]*npz[celltype])) #yeah it's identical cuz one hot encoded is all!

[[ 0.024  0.027 -0.     0.015  0.028  0.     0.     0.     0.     0.026]
 [ 0.     0.    -0.     0.     0.     0.009  0.     0.     0.011  0.   ]
 [ 0.     0.    -0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    -0.     0.     0.     0.     0.031  0.007  0.     0.   ]]
[[1 1 0 1 1 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0]]
True


In [47]:
#now we can loop through all the cell types and save them to the matrix
#we can get the list of files in /data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/ctst* and then loop through them
import os
from tqdm import tqdm
celltype=0
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
for idx, file in tqdm(enumerate(files), total=len(files)):
    if 'ctst' in file and 'reg' in file:
        # print(file)
        # a = np.load('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/'+file)
        ccre = int(file.split('_')[-2])
        # print(ccre)
        out = utils.onehot_ism(ccre, name='multitasking_25epoch')
        npz[idx] = out[0][celltype]
        npz_onehot[idx] = out[1][celltype]

#this is actually goign to be quite slow
#huh, we have to deal with the N somehow

  5%|▍         | 1994/43910 [00:07<02:40, 261.85it/s]


KeyError: 'N'

In [49]:
#let's see now many there are
#now we can loop through all the cell types and save them to the matrix
#we can get the list of files in /data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/ctst* and then loop through them
import os
from tqdm import tqdm
celltype=0
counter=0
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
for file in tqdm(files, total=len(files)):
    if 'ctst' in file and 'reg' in file:
        idx=+1
        # print(file)
        # a = np.load('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/'+file)
        ccre = int(file.split('_')[-2])
        # print(ccre)
        try:
            out = utils.onehot_ism(ccre, name='multitasking_25epoch')
        except:
            counter+=1
            continue
        npz[idx] = out[0][celltype]
        npz_onehot[idx] = out[1][celltype]
print(counter)


100%|██████████| 44110/44110 [04:42<00:00, 156.29it/s]

2


In [50]:
#test the slow part of the code
#we can get the list of files in /data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/ctst* and then loop through them
import os
from tqdm import tqdm
celltype=0
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
for idx, file in tqdm(enumerate(files), total=len(files)):
    if 'ctst' in file and 'reg' in file:
        continue

#This part is literally instant, so this is not the issue at all!

100%|██████████| 44354/44354 [00:00<00:00, 3989067.66it/s]


In [51]:
print(npz_onehot[-3]) #this seems to be fine!

[[1 1 0 ... 0 0 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]


In [53]:
print(npz_onehot[-2]) #this one is all zeros just because we skiped 2 that were N, let's just make it easier on us and delete the one that is N
#that way don't even have to worry about it

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [54]:
import os
from tqdm import tqdm
celltype=0
counter=0
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
for file in tqdm(files, total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx=+1
        # print(file)
        # a = np.load('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/'+file)
        ccre = int(file.split('_')[-2])
        # print(ccre)
        try:
            out = utils.onehot_ism(ccre, name='multitasking_25epoch')
        except:
            print(file)
            counter+=1
            continue
        npz[idx] = out[0][celltype]
        npz_onehot[idx] = out[1][celltype]
print(counter)
#should still be the same 2, nothing changed at all
#the order in which it comes seems to vary as os.listdir just seems to produce a random order each time
#wait wtf?? This isn't right at all, we did check to see if multitasking_25 is in the name comapred to ctst, but it should be finding the same sequences...
#oh, that's actually a huge potential bug!!

#we were dividing by 161 for seqidx, when we don't want to! This is the ccre number!!

 69%|██████▉   | 30613/44480 [03:48<00:57, 241.03it/s]

multitasking_25epoch_752763_reg.npy


100%|██████████| 44480/44480 [05:14<00:00, 141.23it/s]

1


In [55]:
print(npz_onehot[-3]) #so this is loading the same thing but seq won't match it if we did the old ones!!

[[1 1 0 ... 0 0 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]


In [59]:
#wait this doesn't explain the difference,  we should be extracting the exact same stuff. seq should be identical regardless...
#let's just manually load in the sequences and search for an N
for file in tqdm(files, total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        seq = utils.dataset.array[int(file.split('_')[-2]),2]
        # print(seq)
        # break
        if 'N' in seq.upper():
            print(file)
            # break

100%|██████████| 44480/44480 [00:00<00:00, 1570524.81it/s]

multitasking_25epoch_752763_reg.npy


In [61]:
#wait this doesn't explain the difference,  we should be extracting the exact same stuff. seq should be identical regardless...
#let's just manually load in the sequences and search for an N
for file in tqdm(files, total=len(files)):
    if 'ctst' in file and 'reg' in file:
        seq = utils.dataset.array[int(file.split('_')[-2]),2]
        # print(seq)
        # break
        if 'N' in seq.upper():
            print(file)
            print(seq)
            # break
#this one has a lot of N, I support removing it
#the one above with counter == 2 must have run into a separate bug, but glad we fixed the ism bug!!

100%|██████████| 44480/44480 [00:00<00:00, 1708871.63it/s]

ctst_bestepoch_752763_reg.npy
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNccctcttcctgtcacggcccgagactccctcttcctgtcacggcccgagatcccctcttcctgtcacggcccgagactCCCTCTTCCTATCACGGCCCGAGactccctcttcctgtcacggcccgagactccctcttcctgtcacggcccgagaccccctcttcctgtcacggcccgagaccccctcttcctgtcacggcccgagaccccctcttcctgtcacggcccgagaccccctcttcctgtcacggcccgagaccccctcttcctgtcacggcccgagaccCCCTCTTCCAGTCACGGCCCGAGactccctcttcctgtcacggcccgagactccctcttcctgtcacggcccgagactccctcttcctgtcacggcccgagatcccctcttcctgtcacggcctgagttgtttttcaggtttctttgggatCCCCTTGGCTACAAACAGGTCCACTCAGTCAGCTGAGGGGCTTAGAATTCTATTTTTGGTTTACCCCACTATCAGGAGGTTGTCTGAGATAAGCCAGCCCCTCCCACCCTTGCAGGCACAG

In [ ]:
#let's restart the kernel and see if it works, also I did delete this sequence, now we will also compare it between the ctst and multitasking models

# restarting shell and redoing it

In [2]:
#note that we may have some issues with the test/val data, but for now just doing train

from shap_analysis.ism_utils import ISMUtils
multitasking_path1 = '/data/leslie/sarthak/hyena/hyena-dna/outputs/2024-03-27/18-39-11-031863/checkpoints/25-val_loss=0.52186.ckpt' #the 25 epoch one
mult = ISMUtils('DNase_allcelltypes', multitasking_path1, classification=True)
ctst_path = '/data/leslie/sarthak/hyena/hyena-dna/outputs/2024-03-25/15-41-13-286486/checkpoints/last.ckpt'
ctst = ISMUtils('DNase_ctst', ctst_path, classification=True)

In [4]:
#now we can create the matrix, let's just do multitasking first
import numpy as np
npz = np.zeros((16790//2,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot = np.zeros((16790//2,4,1023), dtype=np.int16)

In [18]:
import os
from tqdm import tqdm
celltype=0
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
for file in tqdm(files, total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch')
        # print(out[0].shape)
        # print(out[0][0,:,:10])
        # break
        npz[idx,:,:] = out[0][celltype,:,:]
        npz_onehot[idx,:,:] = out[1][celltype,:,:]
#should still be the same 2, nothing changed at all
#the order in which it comes seems to vary as os.listdir just seems to produce a random order each time
#wait wtf?? This isn't right at all, we did check to see if multitasking_25 is in the name comapred to ctst, but it should be finding the same sequences...
#oh, that's actually a huge potential bug!!

#we were dividing by 161 for seqidx, when we don't want to! This is the ccre number!!

100%|██████████| 47528/47528 [05:02<00:00, 156.95it/s]


In [21]:
print(npz_onehot[0])
np.set_printoptions(precision=3, suppress=True)
print(npz[0])
print(npz_onehot[-1])
print(npz[-1])
print(seq)
#things definitely seem to line up!

[[0 1 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]
[[-0.     0.029  0.062 ...  0.    -0.     0.   ]
 [-0.     0.     0.    ...  0.    -0.002  0.   ]
 [-0.01   0.     0.    ...  0.    -0.     0.   ]
 [-0.     0.     0.    ...  0.003 -0.     0.001]]
[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 1]]
[[-0.     0.048  0.    ...  0.    -0.     0.   ]
 [-0.03   0.     0.    ...  0.    -0.     0.   ]
 [-0.     0.     0.    ...  0.    -0.     0.   ]
 [-0.     0.     0.034 ...  0.001 -0.     0.   ]]
CATGGTTTGAGGAGTGAATAGGAGATGAGAAGGTGCAGAGAGCTAGAAGGAAAGAATTGTAGACATTTTTTACCATGAAGGAAGTTTGAATTACCAGCCTTtcctcattcttttgtatgtaatCACAGAATGATGtcacagtgttttttttgtttttgtttttttattttaccatgtACATATCAACAGATTTTTCTTTGACTATAAAGGACTCTGTGCTTTGATAATAATCCAGAAATGACCAGAGGAAGTTTATCTTGTGTGCACTTTCAATATTGTATTTAAATAGGAAATCACCcctttttacagttttcatcTCGAGCCAAACAAAGTCAAGGTATTATGAGAACTGCAACGCTTTTTGAAGAAACctatcatacatttatttttcttagtttttctttcttcttccctttttaacTGGTTTTGAAGTttggaaataattaaatttt

In [22]:
#now we can do this again for the ctst
npz_ctst = np.zeros((16790//2,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot_ctst = np.zeros((16790//2,4,1023), dtype=np.int16)
idx = -1
for file in tqdm(files, total=len(files)):
    if 'ctst' in file and 'reg' in file:
        idx+=1 #oops, seems that it was =+1 instead of +=1... uh oh 
        ccre = int(file.split('_')[-2])
        seq = ctst.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = ctst.onehot_ism(ccre, name='ctst_bestepoch')
        npz_ctst[idx,:,:] = out[0][celltype,:,:]
        npz_onehot_ctst[idx,:,:] = out[1][celltype,:,:]

100%|██████████| 47528/47528 [05:10<00:00, 153.29it/s]


In [23]:
print(npz_onehot_ctst[0])
np.set_printoptions(precision=3, suppress=True)
print(npz_ctst[0])
print(npz_onehot_ctst[-1])
print(npz_ctst[-1])
print(seq)

[[1 1 0 ... 0 0 0]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]]
[[-0.009 -0.013  0.    ... -0.    -0.    -0.   ]
 [-0.    -0.     0.    ... -0.    -0.    -0.001]
 [-0.    -0.     0.    ... -0.002 -0.    -0.   ]
 [-0.    -0.     0.001 ... -0.    -0.    -0.   ]]
[[1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 1 0 ... 1 0 1]]
[[ 0.016  0.     0.    ...  0.     0.    -0.   ]
 [ 0.     0.     0.006 ...  0.     0.    -0.   ]
 [ 0.     0.     0.    ...  0.     0.001 -0.   ]
 [ 0.     0.047  0.    ...  0.002  0.    -0.001]]
ATCTCCCAAGACAAGTGATGTATTTCCTGAGAATATAGactatgccttttctttcttttctgttcaagTTACACTTAAAATGCTTGTCTTAAAGAATTTTCCTTCTCTTACATGATTTAAGAATGTGAGTGTCTTATATGTAAAACCATATAAGTTACAAGCCATTATAAATGAGAAAGTAGAAATCAGTTATCAATCTCtgattattgaatttttttaagctACTATGTGCAAGTTAAAAGAAAggtgaatttaaaaaagaagaagaaaaaaaaagaggctgggcgcggtggctcacgcctgtaatcccagcactttgggaggccgaggtgggcggatcatgagatgaggagatcgagatcatcctgtctaacacagtgaaaccccatctctactaaaaaatacaaaaaattagctgggcgtagtggcgggcgcctgtagtcccagctact

In [24]:
#now make sure the ctst ones are completely identical
print(np.allclose(npz_ctst,npz_onehot_ctst*npz_ctst))
print(np.allclose(npz,npz_onehot*npz))
print(np.allclose(npz_onehot_ctst,npz_onehot))
#wait they won't ever be in the same order unless we make it so... because things aren't in order!! lmfao!

True
True
False


In [26]:
celltype=0
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
npz = np.zeros((16790//2,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot = np.zeros((16790//2,4,1023), dtype=np.int16)
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch')
        npz[idx,:,:] = out[0][celltype,:,:]
        npz_onehot[idx,:,:] = out[1][celltype,:,:]

#now we can do this again for the ctst
npz_ctst = np.zeros((16790//2,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot_ctst = np.zeros((16790//2,4,1023), dtype=np.int16)
idx = -1
for file in tqdm(sorted(files), total=len(files)):
    if 'ctst' in file and 'reg' in file:
        idx+=1 #oops, seems that it was =+1 instead of +=1... uh oh 
        ccre = int(file.split('_')[-2])
        seq = ctst.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = ctst.onehot_ism(ccre, name='ctst_bestepoch')
        npz_ctst[idx,:,:] = out[0][celltype,:,:]
        npz_onehot_ctst[idx,:,:] = out[1][celltype,:,:]

100%|██████████| 47772/47772 [04:42<00:00, 169.34it/s]


In [27]:
#now make sure the ctst ones are completely identical
print(np.allclose(npz_ctst,npz_onehot_ctst*npz_ctst))
print(np.allclose(npz,npz_onehot*npz))
print(np.allclose(npz_onehot_ctst,npz_onehot))
#now they are in order, so of course they're identical!

True
True
True


In [28]:
#if this works, we will save out the npz files for tfmodisco!!
#then we can create a script to do this approach for all the celltpes simultaneously! The key idea being that we then save all the npz files separately
#but can save it to one massive matrix for some time, should only be like 10 GB, not too bad

#for now save just the ctst files
np.savez_compressed('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/ctst_0_att.npz', npz_ctst)
np.savez_compressed('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/ctst_0_ohe.npz', npz_onehot_ctst)

In [30]:
#now let's visualize them here using HTML!
from IPython.display import display, HTML
HTML('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/report/motifs.html')

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,130,,,TFAP2C_MA0814.1,2.656320e-01,,TFAP2B_MA0812.1,0.265632,,TFAP2A_MA0003.3,0.399194,
pos_patterns.pattern_1,126,,,ZEB1_MA0103.3,1.593940e-01,,HIC2_MA0738.1,0.289703,,CTCFL_MA1102.1,0.289925,
pos_patterns.pattern_2,120,,,PAX9_MA0781.1,1.000000e+00,,PAX1_MA0779.1,1.000000,,NFIC+TLX1_MA0119.1,1.000000,
pos_patterns.pattern_3,91,,,ONECUT3_MA0757.1,8.082870e-02,,ZNF384_MA1125.1,0.080829,,ONECUT2_MA0756.1,0.259860,
pos_patterns.pattern_4,89,,,GLI2_MA0734.1,1.000000e+00,,Hnf4a_MA0114.3,1.000000,,VDR_MA0693.2,1.000000,
pos_patterns.pattern_5,82,,,FOSL1+JUNB_MA1137.1,1.727630e-01,,FOSL1_MA0477.1,0.172763,,FOS+JUN_MA0099.3,0.172763,
pos_patterns.pattern_6,79,,,MEF2D_MA0773.1,4.188980e-01,,ONECUT3_MA0757.1,0.494374,,MEF2A_MA0052.3,0.494374,
pos_patterns.pattern_7,77,,,EWSR1-FLI1_MA0149.1,1.082590e-04,,ZNF263_MA0528.1,0.012941,,E2F6_MA0471.1,0.605273,
pos_patterns.pattern_8,71,,,Six3_MA0631.1,1.000000e+00,,CTCF_MA0139.1,1.000000,,Ar_MA0007.3,1.000000,
pos_patterns.pattern_9,70,,,ZEB1_MA0103.3,1.796030e-01,,PLAG1_MA0163.1,0.558818,,CTCF_MA0139.1,0.558818,


In [31]:
#we're going to try it with a new celltype, this celltype being GM12878, a type of B cell
#importatly, we'll also multiply the results of the ism by -1 for the attributions
#so it's line 55, line 1 is 0 so line 55 is 54
celltype=54
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
npz = np.zeros((16790//2,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot = np.zeros((16790//2,4,1023), dtype=np.int16)
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch')
        npz[idx,:,:] = out[0][celltype,:,:]
        npz_onehot[idx,:,:] = out[1][celltype,:,:]

#now we can do this again for the ctst
npz_ctst = np.zeros((16790//2,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot_ctst = np.zeros((16790//2,4,1023), dtype=np.int16)
idx = -1
for file in tqdm(sorted(files), total=len(files)):
    if 'ctst' in file and 'reg' in file:
        idx+=1 #oops, seems that it was =+1 instead of +=1... uh oh 
        ccre = int(file.split('_')[-2])
        seq = ctst.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = ctst.onehot_ism(ccre, name='ctst_bestepoch')
        npz_ctst[idx,:,:] = out[0][celltype,:,:]
        npz_onehot_ctst[idx,:,:] = out[1][celltype,:,:]

100%|██████████| 48184/48184 [04:53<00:00, 163.98it/s]


In [32]:
print(npz_onehot_ctst[0]) #we can compare this cell to the aboev ones, onehot should be the same, attributions should be different
np.set_printoptions(precision=3, suppress=True)
print(npz_ctst[0])
print(npz_onehot_ctst[-1])
print(npz_ctst[-1])
print(seq)

[[0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 1 1 1]]
[[ 0.     0.    -0.    ... -0.    -0.    -0.   ]
 [ 0.     0.007 -0.01  ... -0.    -0.    -0.   ]
 [ 0.     0.    -0.    ... -0.    -0.    -0.   ]
 [ 0.025  0.    -0.    ... -0.001 -0.002 -0.002]]
[[0 0 0 ... 0 0 0]
 [0 1 1 ... 1 0 1]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 1 0]]
[[ 0.    -0.     0.    ...  0.     0.    -0.   ]
 [ 0.    -0.009  0.016 ...  0.002  0.    -0.001]
 [ 0.    -0.     0.    ...  0.     0.    -0.   ]
 [ 0.013 -0.     0.    ...  0.     0.004 -0.   ]]
TCCCAGCACTCTGCCTTTAATCTTGATTAGTCTAGGTAGATGTTGTTGATCTAGTCACTGAACCGGTTTTTTCTTAAACagttattgaagtataatttacatacaataaaatccaCTCACATTAAGCAtatgatttaataattttttagtaataTTAAAGAGATCTGCAACCATAATCTTGTTTTAGAATAGTTTATCACCTGCAGAAGATGCCTTGTGATAGTTGATAGTTACTTCTGCTTTCACTCcgagccctaggcaaccacttctctactttctgtttctgtatagttatcttttttgacattttctataaatagaataatacaaAATGTAGTCTTTTGAGCCTAGCTTCTTTGACTTAGTGTAAAGTTTTTGTGCTTCATCCTTGTTTTAGTGTGGCTTTGTTAGACTTTAGTATGAGGATAATATTGACCCCataaaatgttttg

In [33]:
#now make sure the ctst ones are completely identical
print(np.allclose(npz_ctst,npz_onehot_ctst*npz_ctst))
print(np.allclose(npz,npz_onehot*npz))
print(np.allclose(npz_onehot_ctst,npz_onehot))
#now they are in order, so of course they're identical!

True
True
True


In [35]:
#save these out too
#and multiplied it by negative 1 here is key, now ctst should be positive
np.savez_compressed('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/ctst_54_att.npz', -npz_ctst)
np.savez_compressed('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/ctst_54_ohe.npz', npz_onehot_ctst)

In [36]:
#now let's visualize them here using HTML!
from IPython.display import display, HTML
HTML('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/report54/motifs.html')

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,637,,,SPIC_MA0687.1,2.158140e-03,,SPI1_MA0080.4,0.010164,,SPIB_MA0081.1,0.096364,
pos_patterns.pattern_1,409,,,IRF1_MA0050.2,1.477770e-04,,STAT1+STAT2_MA0517.1,0.000219,,IRF7_MA0772.1,0.002325,
pos_patterns.pattern_2,345,,,RUNX1_MA0002.2,9.371230e-03,,RUNX3_MA0684.1,0.053451,,RUNX2_MA0511.2,0.053451,
pos_patterns.pattern_3,105,,,CTCF_MA0139.1,8.447220e-12,,CTCFL_MA1102.1,0.000161,,ASCL1_MA1100.1,0.154182,
pos_patterns.pattern_4,68,,,FOSB+JUNB_MA1135.1,6.722310e-06,,FOSL2+JUNB_MA1138.1,0.000007,,FOS+JUN_MA0099.3,0.000007,
pos_patterns.pattern_5,61,,,SPIC_MA0687.1,1.767300e-02,,SPI1_MA0080.4,0.023890,,SPIB_MA0081.1,0.137702,
pos_patterns.pattern_6,43,,,NFKB2_MA0778.1,2.928150e-02,,NFKB1_MA0105.4,0.029281,,RELA_MA0107.1,0.075431,
pos_patterns.pattern_7,34,,,MAX_MA0058.3,5.190650e-01,,ELK4_MA0076.2,0.519065,,BHLHE22_MA0818.1,0.519065,
pos_patterns.pattern_8,28,,,SP2_MA0516.1,2.502390e-03,,SP1_MA0079.3,0.113800,,KLF16_MA0741.1,0.113800,
neg_patterns.pattern_0,319,,,TFAP2C_MA0814.1,2.628440e-01,,TFAP2B_MA0812.1,0.262844,,TFAP2A_MA0003.3,0.404556,


In [37]:
#now do it for 5000 seqlets and compare
HTML('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/report54_5000/motifs.html')

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,701,,,SPIC_MA0687.1,2.269370e-03,,SPI1_MA0080.4,0.010440,,SPIB_MA0081.1,0.095839,
pos_patterns.pattern_1,480,,,IRF1_MA0050.2,2.082510e-04,,STAT1+STAT2_MA0517.1,0.000320,,IRF7_MA0772.1,0.002780,
pos_patterns.pattern_2,394,,,RUNX1_MA0002.2,1.292860e-02,,RUNX3_MA0684.1,0.056087,,RUNX2_MA0511.2,0.056087,
pos_patterns.pattern_3,137,,,CTCF_MA0139.1,8.366720e-12,,CTCFL_MA1102.1,0.000188,,ASCL1_MA1100.1,0.232034,
pos_patterns.pattern_4,78,,,FOSB+JUNB_MA1135.1,1.940080e-05,,FOSL2+JUNB_MA1138.1,0.000019,,FOSL2+JUND_MA1144.1,0.000029,
pos_patterns.pattern_5,51,,,SPIC_MA0687.1,5.813210e-02,,SPI1_MA0080.4,0.106479,,SPIB_MA0081.1,0.326761,
pos_patterns.pattern_6,43,,,TCF3_MA0522.2,6.090780e-02,,SCRT1_MA0743.1,0.060908,,MAX_MA0058.3,0.067281,
pos_patterns.pattern_7,41,,,NFKB2_MA0778.1,3.824260e-02,,NFKB1_MA0105.4,0.038243,,RELA_MA0107.1,0.049401,
pos_patterns.pattern_8,30,,,RELA_MA0107.1,1.549930e-03,,REL_MA0101.1,0.001550,,NFKB2_MA0778.1,0.148465,
pos_patterns.pattern_9,22,,,SNAI2_MA0745.1,9.459630e-02,,TCF3_MA0522.2,0.211084,,FIGLA_MA0820.1,0.233073,


# final steps

In [ ]:
#seems 5000 is better, but doesn't really matter I don't think, the results are basically the same, but now there's more matches
#let's now create a script that can save all the celltypes
#basically, it does it for all the celltypes, then saves it to a massive matrix, then saves out the individual npz files
#first we have to remove the N ones from the files, but it's not bad
#then we basically use the same loop as above, but instead of just one cellttype, can have a large matrix that is 161 x 10k x 4 x 1024
#then we split along the 161 and save it, not bad at all, just do an inner for loop? use smart indexing? unsure, but that's the goal
#then we do for ctst and multitask, not bad at all, use some testing and reuse the above loops, can do some testing by loadin in the old npz files!
#figure out how we want to store/save them! Probably can make a new folder called npz files or soemthing

In [1]:
#note that we may have some issues with the test/val data, but for now just doing train
from shap_analysis.ism_utils import ISMUtils
multitasking_path1 = '/data/leslie/sarthak/hyena/hyena-dna/outputs/2024-03-27/18-39-11-031863/checkpoints/25-val_loss=0.52186.ckpt' #the 25 epoch one
mult = ISMUtils('DNase_allcelltypes', multitasking_path1, classification=True)
ctst_path = '/data/leslie/sarthak/hyena/hyena-dna/outputs/2024-03-25/15-41-13-286486/checkpoints/last.ckpt'
ctst = ISMUtils('DNase_ctst', ctst_path, classification=True)

In [7]:
#first check to see if in val and test have any with N in it
#wait this doesn't explain the difference,  we should be extracting the exact same stuff. seq should be identical regardless...
#let's just manually load in the sequences and search for an N
import os
from tqdm import tqdm
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/val/')
for file in tqdm(files, total=len(files)):
    if 'ctst' in file and 'reg' in file:
        seq = ctst.dataset.array[int(file.split('_')[-2]),2]
        # print(seq)
        # break
        if 'N' in seq.upper():
            print(file)
            print(seq)

files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/test/')
for file in tqdm(files, total=len(files)):
    if 'ctst' in file and 'reg' in file:
        seq = ctst.dataset.array[int(file.split('_')[-2]),2]
        # print(seq)
        # break
        if 'N' in seq.upper():
            print(file)
            print(seq)
            # break
#this one has a lot of N, I support removing it
#the one above with counter == 2 must have run into a separate bug, but glad we fixed the ism bug!!

100%|██████████| 3096/3096 [00:00<00:00, 1267626.43it/s]


In [14]:
#now let's do it for all 3 files and also all celltypes simultaneously
#we have 9999 files!
import numpy as np
idx=-1
celltype = 5 #we will test it with the old celltype approach to see
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
npz_old = np.zeros((9999,4,1023))
npz = np.zeros((161,9999,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot = np.zeros((161,9999,4,1023), dtype=np.int16)
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch')
        # print(out[0].shape)
        # break
        # npz[idx,:,:] = out[0][celltype,:,:]
        # npz_onehot[idx,:,:] = out[1][celltype,:,:]
        npz_old[idx,:,:] = out[0][celltype,:,:]
        npz[:,idx] = out[0]
        npz_onehot[:,idx] = out[1]
        if idx == 1000:
            break

  0%|          | 0/48184 [00:00<?, ?it/s]

 39%|███▉      | 18791/48184 [00:24<00:39, 753.39it/s]   


In [15]:
#now we test to make sure they're the same
np.allclose(npz[celltype], npz_old)

True

In [3]:
#let's also check the size of the matrix
import numpy as np
npz = np.zeros((161,9999,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz.nbytes/(1024**3)



12.270102627575397

In [4]:
#this is perfect, let's do it but entire thing now
#now let's do it for all 3 files and also all celltypes simultaneously
#we have 9999 files!
import numpy as np
import os
from tqdm import tqdm
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
# npz_old = np.zeros((9999,4,1023))
npz = np.zeros((161,9999,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_onehot = np.zeros((161,9999,4,1023), dtype=np.int16)
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch')
        npz[:,idx] = out[0]
        npz_onehot[:,idx] = out[1]

files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/val/')
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch', split='val')
        npz[:,idx] = out[0]
        npz_onehot[:,idx] = out[1]

files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/test/')
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = mult.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = mult.onehot_ism(ccre, name='multitasking_25epoch', split = 'test')
        npz[:,idx] = out[0]
        npz_onehot[:,idx] = out[1]

100%|██████████| 3096/3096 [00:37<00:00, 81.90it/s]   


In [5]:
#let's save this out then delete them
file_path = '/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/cell_data/'
for cell_idx in range(161):
    np.savez_compressed(file_path+'multitasking_'+str(cell_idx)+'_att.npz', npz[cell_idx])
    np.savez_compressed(file_path+'multitasking_'+str(cell_idx)+'_ohe.npz', npz_onehot[cell_idx])

del npz

In [3]:
#this is perfect, let's do it but entire thing now
#now let's do it for all 3 files and also all celltypes simultaneously
#we have 9999 files!
import os
from tqdm import tqdm
import numpy as np
idx=-1
files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/train/')
npz_ctst = np.zeros((161,9999,4,1023), dtype=np.float16) #reduced the size by 1, because removing that one
npz_ctst_onehot = np.zeros((161,9999,4,1023), dtype=np.int16)
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = ctst.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = ctst.onehot_ism(ccre, name='ctst_bestepoch')
        npz_ctst[:,idx] = out[0]
        npz_ctst_onehot[:,idx] = out[1]

files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/val/')
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = ctst.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = ctst.onehot_ism(ccre, name='ctst_bestepoch', split='val')
        npz_ctst[:,idx] = out[0]
        npz_ctst_onehot[:,idx] = out[1]

files = os.listdir('/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/most_variable_cCREs2/test/')
for file in tqdm(sorted(files), total=len(files)):
    if 'multitasking_25' in file and 'reg' in file:
        idx+=1
        ccre = int(file.split('_')[-2])
        seq = ctst.dataset.array[ccre,2]
        if 'N' in seq.upper():
            print(file)
            break
        out = ctst.onehot_ism(ccre, name='ctst_bestepoch', split = 'test')
        npz_ctst[:,idx] = out[0]
        npz_ctst_onehot[:,idx] = out[1]

100%|██████████| 3096/3096 [00:35<00:00, 87.49it/s]   


In [7]:
#there's one test we can do
np.allclose(npz_ctst,npz_ctst_onehot*npz_ctst)
#this will probalby crash it tho
#oh it idnd't crash, well this is perfect!

True

In [5]:
#now we can save them out again
file_path = '/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/cell_data/'
for cell_idx in range(161):
    np.savez_compressed(file_path+'ctst_'+str(cell_idx)+'_att.npz', npz_ctst[cell_idx])
    np.savez_compressed(file_path+'ctst_'+str(cell_idx)+'_ohe.npz', npz_ctst_onehot[cell_idx])

In [6]:
#now make sure the ohe are the same
#we can do this by loading them in one at a time and seeing if it's true
for i in range(161):
    a = np.load(file_path+'ctst_'+str(i)+'_ohe.npz')['arr_0']
    b = np.load(file_path+'multitasking_'+str(i)+'_ohe.npz')['arr_0']
    print(temp:=np.allclose(a,b)) #should be true
    if temp is False:
        break
#enough of them are true for me to just say it's fine! now we are done with this part!

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

In [ ]:
#now we save them out in an npz file, base it on the same naming scheme
file_path = '/data/leslie/sarthak/hyena/hyena-dna/shap_analysis/modisco/all_celltypes/cell_data/'
for cell_idx in range(161):
    np.savez_compressed(file_path+'multitasking_'+str(cell_idx)+'_att.npz', npz[cell_idx])
    np.savez_compressed(file_path+'multitasking_'+str(cell_idx)+'_ohe.npz', npz_onehot[cell_idx])
    np.savez_compressed(file_path+'ctst_'+str(cell_idx)+'_att.npz', npz_ctst[cell_idx])
    np.savez_compressed(file_path+'ctst_'+str(cell_idx)+'_ohe.npz', npz_ctst_onehot[cell_idx])